In [65]:
1 * 3

3

In [63]:
1 + 1

2

# Load and connect data

In [35]:
import mage_ai

mage_ai.launch(host='127.0.0.1', port=1234)

Mage running on host and port 127.0.0.1:1234 * Serving Flask app 'mage_ai.server.app' (lazy loading)

Open UI in another tab with url: http://127.0.0.1:1234
 * Environment: production


   Use a production WSGI server instead.


<ThreadWithTrace(Thread-5, started daemon 13135855616)>

 * Debug mode: off


In [1]:
%connect_info

{
  "shell_port": 61139,
  "iopub_port": 61140,
  "stdin_port": 61141,
  "control_port": 61143,
  "hb_port": 61142,
  "ip": "127.0.0.1",
  "key": "45d97f1c-db336e97477eea00e0acf957",
  "transport": "tcp",
  "signature_scheme": "hmac-sha256",
  "kernel_name": ""
}

Paste the above JSON into a file, and connect with:
    $> jupyter <app> --existing <file>
or, if you are local, you can connect with just:
    $> jupyter <app> --existing kernel-bfe3ad2d-fcbc-40f4-b4b2-0bdb1705bfab.json
or even just:
    $> jupyter <app> --existing
if this is the most recent Jupyter kernel you have started.


In [4]:
%pip install mysql-connector-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 342.0/342.0 kB 2.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 403.3/403.3 kB 7.2 MB/s eta 0:00:0000:01
Note: you may need to restart the kernel to use updated packages.


In [1]:
import mysql.connector
import os
import pandas as pd


def query(sql):
    mydb = mysql.connector.connect(
      host='mage-development.cxj4djmtpwkx.us-west-2.rds.amazonaws.com',
      user='root',
      password=os.getenv('DB_PASSWORD'),
      database='materia_development',
    )
    mycursor = mydb.cursor()

    mycursor.execute(sql)

    return mycursor.fetchall()


def query_table(table_name):
    columns = [r[0] for r in query(f'DESCRIBE {table_name}')]
    rows = query(f'SELECT * FROM {table_name}')
    
    return pd.DataFrame(rows, columns=columns)

ModuleNotFoundError: No module named 'mysql'

In [2]:
', '.join([r[0] for r in query('SHOW TABLES')])

ProgrammingError: 1045 (28000): Access denied for user 'root'@'c-98-207-95-145.hsd1.ca.comcast.net' (using password: NO)

In [78]:
import mage_ai

# df = query_table('materia_development.model_with_versions')
# mage_ai.connect_data(df, 'model_with_versions')

df = query_table('materia_development.features_with_feature_sets')
mage_ai.connect_data(df, 'features_with_feature_sets')

<FeatureSet 35> Cleaning suggestions:
1. Remove columns with no values(['partition_filter_version']): Remove columns with no values to increase data quality.
2. Remove columns with high empty rate(['deleted_at_feature', 'window_feature', 'transformer_action_id_feature', 'deleted_at_version', 'parent_feature_set_version_id_version', 'reference_feature_set_version_id_version', 'insights_analyzed_at_version', 'deleted_at_feature_set', 'description_feature_set', 'source_type_feature_set']): Remove columns with many missing values may increase data quality.
3. Remove columns with single value(['structure_type_version']): Remove columns with a single unique value to reduce the amount of redundant data.
4. Fix syntax errors(['username_user']): Fix syntactical errors to reduce the amount of noise in the data.
5. Reformat values(['username_user', 'email_user', 'uuid_feature', 'name_feature', 'description_feature', 'name_feature_set']): Format entries in these columns as fully lowercase to impro

In [4]:
import mage_ai
from mage_ai.sample_datasets import load_dataset


df = load_dataset('titanic_survival.csv')
mage_ai.connect_data(df, name='titanic dataset')

<FeatureSet 37> Cleaning suggestions:
1. Clean dirty column names(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked']): Format these column names as lowercase and alphanumeric to improve accessibility and security.
2. Reformat values(['Name']): Format entries in these columns as fully lowercase to improve data quality.
3. Fill in missing values(['Age']): For each column, fill missing entries with the average value.
4. Fill in missing values(['Cabin']): Fill missing values with a placeholder to mark them as missing.
5. Fill in missing values(['Embarked']): For each column, fill missing entries with the most frequent value.
6. Remove outliers(['Age']): Remove 2 outlier(s) to reduce the amount of noise in this column.
7. Remove outliers(['SibSp']): Remove 30 outlier(s) to reduce the amount of noise in this column.
8. Remove outliers(['Parch']): Remove 15 outlier(s) to reduce the amount of noise in this column.
9. Remove outl

In [ ]:
mage_ai.launch()

# Clean data using pipeline created from app

In [5]:
mage_ai.clean(df, pipeline_uuid=36)

,PassengerId,Survived,Pclass,Name,Sex,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,1,0,A/5 21171,7.2500,missing,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,0,0,STON/O2. 3101282,7.9250,missing,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,0,0,373450,8.0500,missing,S
...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,0,0,211536,13.0000,missing,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,1,2,W./C. 6607,23.4500,missing,S
889,890,1,1,"Behr, Mr. Karl Howell",male,0,0,111369,30.0000,C148,C


In [34]:
1 + 1

2